In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

from temp import NeuralPix2Vid
from neuralop.models import FNO

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from temp import check_torch
device = check_torch()

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.1.2+cu118 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): Quadro M6000 24GB
------------------------------------------------------------


In [2]:
sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz, nz_short  = 100, 100, 11, 5

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
Tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']
print('Grid: {} | Tops: {}'.format(Grid.shape, Tops.shape))

Grid_short = Grid[:,:,5:10]
Grid_ext = np.repeat(np.expand_dims(Grid, 0), 33, axis=0)
Grid_short_ext = np.repeat(np.expand_dims(Grid_short, 0), 33, axis=0)
print('Grid_ext: {} | Grid_short_ext: {}'.format(Grid_ext.shape, Grid_short_ext.shape))

Grid: (100, 100, 11) | Tops: (100, 100, 11)
Grid_ext: (33, 100, 100, 11) | Grid_short_ext: (33, 100, 100, 5)


In [3]:
train_size = 50

train_idx = np.random.choice(range(1272), size=train_size, replace=False)
test_idx  = np.setdiff1d(range(1272), train_idx)

xm = np.zeros((len(train_idx), 3, 100,100,5))
xw = np.zeros((len(train_idx), 2, 5))
xc = np.zeros((len(train_idx), n_timesteps, 5))
xt = np.zeros((len(train_idx), n_timesteps, 1))
yy = np.zeros((len(train_idx), 33, 2, 100,100,5))

def apply_mask(x, imap=indexMap, mask_value=0.0):
    xx = mask_value*np.ones((nx,ny,nz)).flatten(order='F')
    xx[imap] = x.flatten(order='F')[imap]
    xx = xx.reshape((nx,ny,nz), order='F')
    return xx

for i in range(len(train_idx)):
    m = np.load('data_npy_100_100_11/inputs_rock_rates_locs_time/x_{}.npz'.format(train_idx[i]))
    p = np.expand_dims(apply_mask(m['poro']), 0)[...,5:10] / 0.3
    k = np.expand_dims(apply_mask(m['perm']), 0)[...,5:10] / 3.3
    t = np.expand_dims(apply_mask(Tops), 0)[...,5:10]      / Tops.max()
    xm[i] = np.concatenate([t, p, k], 0)

    xw[i] = m['locs']
    xc[i] = m['ctrl']
    xt[i] = m['time']

    dd = np.load('data_npy_100_100_11/outputs_pressure_saturation/y_{}.npz'.format(train_idx[i]))
    prm = dd['pressure'][...,5:10]
    sam = dd['saturation'][...,5:10]
    yy[i,:,0] = np.expand_dims(prm, 0)
    yy[i,:,1] = np.expand_dims(sam, 0)

In [4]:
x_m_tensor = torch.tensor(xm, dtype=torch.float32)
x_w_tensor = torch.tensor(xw, dtype=torch.float32).reshape(train_size, -1)
x_c_tensor = torch.tensor(xc, dtype=torch.float32).reshape(train_size, -1)
x_t_tensor = torch.tensor(xt, dtype=torch.float32).reshape(train_size, -1)
y_tensor = torch.tensor(yy, dtype=torch.float32)
print('xm', x_m_tensor.shape)
print('xw', x_w_tensor.shape)
print('xc', x_c_tensor.shape)
print('xt', x_t_tensor.shape)
print('yy', y_tensor.shape)

xm torch.Size([50, 3, 100, 100, 5])
xw torch.Size([50, 10])
xc torch.Size([50, 165])
xt torch.Size([50, 33])
yy torch.Size([50, 33, 2, 100, 100, 5])


In [5]:
train_dataset = TensorDataset(x_m_tensor[:-10], x_w_tensor[:-10], 
                              x_c_tensor[:-10], x_t_tensor[:-10], 
                              y_tensor[:-10])

valid_dataset = TensorDataset(x_m_tensor[-10:], x_w_tensor[-10:], 
                              x_c_tensor[-10:], x_t_tensor[-10:], 
                              y_tensor[-10:])

trainloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=10, shuffle=False)

In [6]:
from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIMLoss
from torchmetrics.image import SpatialCorrelationCoefficient as SCCLoss
from torchmetrics.image import SpectralDistortionIndex as SDILoss
from torchmetrics.image import UniversalImageQualityIndex as UIQILoss
from torchmetrics.image import VisualInformationFidelity as VIFLoss

c:\ProgramData\Anaconda3\envs\torchy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class CustomLoss(nn.Module):
    def __init__(self, alpha=0.8, beta=0.7):
        super(CustomLoss, self).__init__()
        self.ssim = SSIMLoss()
        self.mse  = nn.MSELoss()
        self.mae  = nn.L1Loss()
        self.alpha = alpha
        self.beta  = beta

    def forward(self, true, pred):
        ridge_loss = self.beta*self.mse(true, pred) + (1-self.beta)*self.mae(true, pred)
        visual_loss = 1-self.ssim(true, pred)
        total_loss = self.alpha*ridge_loss + (1-self.alpha)*visual_loss
        return total_loss

In [8]:
model = NeuralPix2Vid().to(device)
print('# parameters: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = CustomLoss().to(device)

# parameters: 34,631,418


In [9]:
epochs, monitor = 101, 10
train_loss, valid_loss = [], []
for epoch in range(epochs):
    # training
    epoch_train_loss = []
    model.train()
    for i, (xm, xw, xc, xt, y) in enumerate(trainloader):
        optimizer.zero_grad()
        xm = xm.to(device)
        xw = xw.to(device)
        xc = xc.to(device)
        xt = xt.to(device)
        yt = y.to(device)
        yh = model(xm, xw, xc, xt).to(device)
        loss = criterion(yt, yh)
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))
    # validation
    model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        for i, (xmv, xwv, xcv, xtv, yv) in enumerate(validloader):
            xmv = xmv.to(device)
            xwv = xwv.to(device)
            xcv = xcv.to(device)
            xtv = xtv.to(device)
            yv = yv.to(device)
            yhv = model(xmv, xwv, xcv, xtv).to(device)
            loss = criterion(yv, yhv)
            epoch_valid_loss.append(loss.item())
    valid_loss.append(np.mean(epoch_valid_loss))
    # print
    if epoch % monitor == 0:
        print('Epoch: [{}/{}] | Loss: {:.4f} | Valid Loss: {:.4f}'.format(
            epoch+1, epochs, train_loss[-1], valid_loss[-1]))

torch.save(model.state_dict(), 'neuralpix2vid.pth')
pd.DataFrame({'train': train_loss, 'valid': valid_loss}).to_csv('neuralpix2vid_losses.csv', index=False)

RuntimeError: Padding size should be less than the corresponding input dimension, but got: padding (5, 5) at dimension 2 of input [10, 33, 2, 50000]

***
# END